## Preperation: load dependencies

In [1]:
//import $ivy.`com.typesafe.akka::akka-http-spray-json:10.2.4`
//import $ivy.`com.typesafe.slick::slick:3.3.3`
//import $ivy.`com.typesafe.slick::slick-hikaricp:3.3.3`
//import $ivy.`com.github.tminglei::slick-pg:0.19.5`
//import $ivy.`com.github.tminglei::slick-pg_spray-json:0.19.5`
//import $ivy.`org.postgresql:postgresql:42.2.19`
//import $ivy.`ch.qos.logback:logback-classic:1.2.3`

// Load fat JAR
interp.load.cp(os.Path("/home/jovyan/jar/masterthesis-khipu-assembly-0.1.jar"))

In [2]:
// Smile

import $ivy.`com.github.haifengl::smile-scala:2.6.0`

import scala.language.postfixOps
import org.apache.commons.csv.CSVFormat
import smile._
import smile.util._
import smile.math._
import smile.math.MathEx._
import smile.math.kernel._
import smile.math.matrix._
import smile.math.matrix.Matrix._
import smile.math.rbf._
import smile.stat.distribution._
import smile.data._
import smile.data.formula._
import smile.data.measure._
import smile.data.`type`._
import smile.json._
import smile.interpolation._
import smile.validation._
import smile.association._
import smile.base.cart.SplitRule
import smile.base.mlp._
import smile.base.rbf.RBF
import smile.classification._
import smile.regression._
import smile.feature._
import smile.clustering._
import smile.vq._
import smile.manifold._
import smile.mds._
import smile.sequence._
import smile.projection._
import smile.nlp._
import smile.wavelet._
// Plotting
import smile.plot.swing._
import smile.plot.vega._
import smile.plot.show
import smile.plot.Render._

import $ivy.$                                       


import scala.language.postfixOps

import org.apache.commons.csv.CSVFormat

import smile._

import smile.util._

import smile.math._

import smile.math.MathEx._

import smile.math.kernel._

import smile.math.matrix._

import smile.math.matrix.Matrix._

import smile.math.rbf._

import smile.stat.distribution._

import smile.data._

import smile.data.formula._

import smile.data.measure._

import smile.data.`type`._

import smile.json._

import smile.interpolation._

import smile.validation._

import smile.association._

import smile.base.cart.SplitRule

import smile.base.mlp._

import smile.base.rbf.RBF

import smile.classification._

import smile.regression._

import smile.feature._

import smile.clustering._

import smile.vq._

import smile.manifold._

import smile.mds._

import smile.sequence._

import smile.projection._

import smile.nlp._

import smile.wavelet._
// Plotting

import smile.plot.swing._

import smile.plot.vega._

i

## Get data from DB

In [6]:
import scala.concurrent.ExecutionContext.Implicits.global
import scala.concurrent.Future
import scala.concurrent.duration._
import scala.concurrent.{ Await, Future }
import scala.language.postfixOps

import de.tmrdlt.database.workflowlist.WorkflowListDB
import de.tmrdlt.database.action.ActionDB


val workflowListDB = new WorkflowListDB()
val actionDB = new ActionDB()

val workflowLists =  Await.result(workflowListDB.getWorkflowLists, 2 seconds)
val actions =  Await.result(actionDB.getActions, 2 seconds)

import scala.concurrent.ExecutionContext.Implicits.global

import scala.concurrent.Future

import scala.concurrent.duration._

import scala.concurrent.{ Await, Future }

import scala.language.postfixOps


import de.tmrdlt.database.workflowlist.WorkflowListDB

import de.tmrdlt.database.action.ActionDB



workflowListDB: WorkflowListDB = de.tmrdlt.database.workflowlist.WorkflowListDB@1d96128a
actionDB: ActionDB = de.tmrdlt.database.action.ActionDB@78d5b271
workflowLists: Seq[de.tmrdlt.database.workflowlist.WorkflowList] = Vector(
  WorkflowList(
    1L,
    "4f90a6d98f77505d7940ce88",
    "Brackets",
    Some(
      "An open source code editor for the web. These are the features we're planning to build."
    ),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2012-04-20T01:59:21,
    2021-02-26T22:24:55.286
  ),
  WorkflowList(
    2L,
    "4f99afb90c8915c46e1bbe4d",
    "Icebox (To Be Reviewed)",
    None,
    Some(1L),
    0L,
    LIST,
    Some(OPEN),
    Trello,

# Trello
## Size of boards and columns over time

In [8]:
import de.tmrdlt.models._
import java.time.LocalDate
import scala.collection.mutable.ListBuffer

val boards = workflowLists
    .filter(_.listType == WorkflowListType.BOARD)
    .filter(_.dataSource == WorkflowListDataSource.Trello)

import de.tmrdlt.models._

import java.time.LocalDate

import scala.collection.mutable.ListBuffer


boards: Seq[de.tmrdlt.database.workflowlist.WorkflowList] = Vector(
  WorkflowList(
    1L,
    "4f90a6d98f77505d7940ce88",
    "Brackets",
    Some(
      "An open source code editor for the web. These are the features we're planning to build."
    ),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2012-04-20T01:59:21,
    2021-02-26T22:24:55.286
  ),
  WorkflowList(
    917L,
    "5fad0555e165558a64ad0478",
    "Masterarbeit",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-12T13:44:20.759
  ),
  WorkflowList(
    955L,
    "5fad0555e165558a64ad0478",
    "Masterarbeit",
    Some(""),
    None,
    0L,
...

In [12]:
val board = boards(1)

val data = new ListBuffer[(Int, LocalDate)]
data += ((0, board.createdAt.toLocalDate))

actions
    .filter(a => (a.actionType == "createCard" || a.actionType == "deleteCard") 
           && a.boardApiId == Some(board.apiId))
    .sortBy(_.date)
    .foreach { a => 
        val count = if (a.actionType == "createCard") data.last._1 + 1 else data.last._1 - 1
        data += ((count, a.date.toLocalDate))
    }

val smileData = data.toList.map(d => JsObject("date" -> JsDate(d._2), "count" -> JsInt(d._1)))
val chart = VegaLite.view()
    .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
    .width(800)
    .height(200)
    .x(field = "date", `type` = "temporal")
    .y(field = "count", `type` = "quantitative")
    .data(smileData)
    .title(s"Size of the board '${board.title}' over time")
show(chart)

val columns = workflowLists
.filter(_.listType == WorkflowListType.LIST)
.filter(_.parentId == Some(board.id))
.sortBy(_.position)
val columnData = 
columns.map { column =>
    val data = new ListBuffer[(Int, LocalDate, String, Int)]
    data += ((0, column.createdAt.toLocalDate, column.title, column.position.toInt))
    
    actions
        .filter(a => a.parentApiId == Some(column.apiId) ||
                     a.oldParentApiId == Some(column.apiId) ||
                     a.newParentApiId == Some(column.apiId))
        .sortBy(_.date)
        .foreach { a => 
            val count = 
                if (a.actionType == "createCard") {
                    data.last._1 + 1
                } else if (a.actionType == "deleteCard") {
                    data.last._1 - 1  
                } else {
                    if (column.apiId == a.oldParentApiId.get) {
                        data.last._1 - 1
                    } else {
                        data.last._1 + 1
                    }
                } 
            data += ((count, a.date.toLocalDate, column.title, column.position.toInt))
        }
    data
}

columnData.foreach { data =>
    val smileData = data.toList.map(d => JsObject("date" -> JsDate(d._2), "count" -> JsInt(d._1)))
    val chart = VegaLite.view()
        .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
        .width(800)
        .height(200)
        .x(field = "date", `type` = "temporal")
        .y(field = "count", `type` = "quantitative")
        .data(smileData)
        .title(s"Size of column '${data(0)._3}' over time")
    show(chart)
}
// Plot as Facet
//val columnSmileData = columnData.flatten.toList.map(d => JsObject("card_count" -> JsInt(d._1), "date" -> JsDate(d._2), "column_name" -> JsString(d._3), "position" -> JsInt(d._4)))
//val chart2 = VegaLite.facet(columnSmileData)
//    .row(field = "column_name", `type` = "nominal")
//    .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
//    .width(800)
//    .height(150)
//    .x(field = "date", `type` = "temporal")
//    .y(field = "card_count", `type` = "quantitative")
//    .title(s"Size of the columns of board '${board.title}' over time")
//show(chart2)

<iframe id="ea931f63-6ae7-4482-8f50-833b9efe35bf" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2020-11-12","count":0},{"date":"2020-11-12","count":1},{"date":"2020-11-12","count":2},{"date":"2020-11-12","count":3},{"date":"2020-11-12","count":4},{"date":"2020-11-12","count":5},{"date":"2020-11-12","count":6},{"date":"2020-11-12","count":7},{"date":"2020-11-12","count":8},{"date":"2020-11-12","count":9},{"date":"2020-11-12","count":10},{"date":"2020-11-12","count":11},{"date":"2020-11-12","count":12},{"date":"2020-11-30","count":13},{"date":"2020-11-30","count":14},{"date":"2020-11-30","count":15},{"date":"2020-11-30","count":16},{"date":"2020-11-30","count":17},{"date":"2020-11-30","count":18},{"date":"2020-11-30","count":19},{"date":"2020-11-30","count":20},{"date":"2020-11-30","count":21},{"date":"2020-11-30","count":22},{"date":"2020-11-30","count":23},{"date":"2020-11-30","count":24},{"date":"2020-11-30","count":25},{"date":"2020-11-30","count":26},{"date":"2020-11-30","count":27},{"date":"2020-12-01","count":28},{"date":"2020-12-01","count":29},{"date":"2020-12-01","count":30},{"date":"2020-12-01","count":31},{"date":"2020-12-01","count":32},{"date":"2020-12-01","count":33},{"date":"2020-12-01","count":34},{"date":"2020-12-01","count":35},{"date":"2020-12-01","count":36},{"date":"2020-12-01","count":37},{"date":"2020-12-01","count":38},{"date":"2020-12-01","count":39},{"date":"2020-12-04","count":40},{"date":"2020-12-04","count":41},{"date":"2020-12-04","count":42},{"date":"2020-12-04","count":43},{"date":"2020-12-04","count":44},{"date":"2020-12-04","count":45},{"date":"2020-12-07","count":44},{"date":"2020-12-07","count":43},{"date":"2020-12-07","count":42},{"date":"2020-12-07","count":41},{"date":"2020-12-07","count":40},{"date":"2020-12-07","count":39},{"date":"2020-12-07","count":40},{"date":"2020-12-07","count":41},{"date":"2020-12-07","count":42},{"date":"2020-12-08","count":43},{"date":"2020-12-08","count":44},{"date":"2020-12-08","count":45},{"date":"2020-12-14","count":46},{"date":"2020-12-14","count":47},{"date":"2020-12-14","count":48},{"date":"2021-01-25","count":49},{"date":"2021-01-25","count":50},{"date":"2021-01-25","count":51},{"date":"2021-02-02","count":52},{"date":"2021-02-02","count":53},{"date":"2021-02-02","count":54},{"date":"2021-02-02","count":55},{"date":"2021-02-02","count":56},{"date":"2021-02-02","count":57},{"date":"2021-02-02","count":58},{"date":"2021-02-02","count":59},{"date":"2021-02-02","count":60},{"date":"2021-02-09","count":61},{"date":"2021-02-09","count":62},{"date":"2021-02-09","count":63},{"date":"2021-02-15","count":64},{"date":"2021-02-15","count":65},{"date":"2021-02-15","count":66},{"date":"2021-02-15","count":67},{"date":"2021-02-15","count":68},{"date":"2021-02-15","count":69},{"date":"2021-02-22","count":70},{"date":"2021-02-22","count":71},{"date":"2021-02-22","count":72},{"date":"2021-02-23","count":73},{"date":"2021-02-23","count":74},{"date":"2021-02-23","count":75},{"date":"2021-02-26","count":76},{"date":"2021-02-26","count":77},{"date":"2021-02-26","count":78},{"date":"2021-03-08","count":79},{"date":"2021-03-08","count":80},{"date":"2021-03-08","count":81},{"date":"2021-03-11","count":80},{"date":"2021-03-11","count":79},{"date":"2021-03-11","count":78},{"date":"2021-03-11","count":79},{"date":"2021-03-11","count":80},{"date":"2021-03-11","count":81}]},"width":800,"title":"Size of the board 'Masterarbeit' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"typ

<iframe id="ba1dc887-2bbb-4627-ba47-7309c5dbded3" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2020-11-12","count":0},{"date":"2020-11-12","count":1},{"date":"2020-11-12","count":2},{"date":"2020-11-12","count":3},{"date":"2020-11-12","count":4},{"date":"2020-11-12","count":5},{"date":"2020-11-12","count":6},{"date":"2020-11-12","count":7},{"date":"2020-11-12","count":8},{"date":"2020-11-12","count":9},{"date":"2020-11-12","count":10},{"date":"2020-11-12","count":11},{"date":"2020-11-12","count":12},{"date":"2020-11-30","count":13},{"date":"2020-11-30","count":14},{"date":"2020-11-30","count":15},{"date":"2020-11-30","count":14},{"date":"2020-11-30","count":13},{"date":"2020-11-30","count":12},{"date":"2020-11-30","count":11},{"date":"2020-11-30","count":10},{"date":"2020-11-30","count":9},{"date":"2020-11-30","count":8},{"date":"2020-11-30","count":7},{"date":"2020-11-30","count":6},{"date":"2020-11-30","count":7},{"date":"2020-11-30","count":8},{"date":"2020-11-30","count":9},{"date":"2020-11-30","count":10},{"date":"2020-11-30","count":11},{"date":"2020-11-30","count":12},{"date":"2020-12-01","count":13},{"date":"2020-12-01","count":14},{"date":"2020-12-01","count":15},{"date":"2020-12-01","count":16},{"date":"2020-12-01","count":17},{"date":"2020-12-01","count":18},{"date":"2020-12-01","count":17},{"date":"2020-12-01","count":16},{"date":"2020-12-01","count":15},{"date":"2020-12-01","count":14},{"date":"2020-12-01","count":13},{"date":"2020-12-01","count":12},{"date":"2020-12-04","count":13},{"date":"2020-12-04","count":14},{"date":"2020-12-04","count":15},{"date":"2020-12-04","count":14},{"date":"2020-12-04","count":13},{"date":"2020-12-04","count":12},{"date":"2020-12-04","count":11},{"date":"2020-12-04","count":10},{"date":"2020-12-04","count":9},{"date":"2020-12-04","count":8},{"date":"2020-12-04","count":7},{"date":"2020-12-04","count":6},{"date":"2020-12-07","count":5},{"date":"2020-12-07","count":4},{"date":"2020-12-07","count":3},{"date":"2020-12-07","count":2},{"date":"2020-12-07","count":1},{"date":"2020-12-07","count":0},{"date":"2020-12-07","count":-1},{"date":"2020-12-07","count":-2},{"date":"2020-12-07","count":-3},{"date":"2020-12-07","count":-2},{"date":"2020-12-07","count":-1},{"date":"2020-12-07","count":0},{"date":"2020-12-08","count":1},{"date":"2020-12-08","count":2},{"date":"2020-12-08","count":3},{"date":"2020-12-08","count":2},{"date":"2020-12-08","count":1},{"date":"2020-12-08","count":0},{"date":"2020-12-14","count":1},{"date":"2020-12-14","count":2},{"date":"2020-12-14","count":3},{"date":"2020-12-14","count":4},{"date":"2020-12-14","count":5},{"date":"2020-12-14","count":6},{"date":"2020-12-14","count":7},{"date":"2020-12-14","count":8},{"date":"2020-12-14","count":9},{"date":"2020-12-14","count":10},{"date":"2020-12-14","count":11},{"date":"2020-12-14","count":12},{"date":"2020-12-14","count":11},{"date":"2020-12-14","count":10},{"date":"2020-12-14","count":9},{"date":"2020-12-14","count":10},{"date":"2020-12-14","count":11},{"date":"2020-12-14","count":12},{"date":"2020-12-14","count":13},{"date":"2020-12-14","count":14},{"date":"2020-12-14","count":15},{"date":"2020-12-14","count":16},{"date":"2020-12-14","count":17},{"date":"2020-12-14","count":18},{"date":"2021-01-25","count":19},{"date":"2021-01-25","count":20},{"date":"2021-01-25","count":21},{"date":"2021-02-09","count":22},{"date":"2021-02-09","count":23},{"date":"2021-02-09","count":24},{"date":"2021-02-15","count":25},{"date":"2021-02-15","count":26},{"date":"2021-02-15","count":27},{"date":"20

<iframe id="d8f209cb-6b19-4088-8065-46625656cdc4" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2020-11-12","count":0},{"date":"2020-11-30","count":1},{"date":"2020-11-30","count":2},{"date":"2020-11-30","count":3},{"date":"2020-11-30","count":4},{"date":"2020-11-30","count":5},{"date":"2020-11-30","count":6},{"date":"2020-11-30","count":7},{"date":"2020-11-30","count":8},{"date":"2020-11-30","count":9},{"date":"2020-12-01","count":10},{"date":"2020-12-01","count":11},{"date":"2020-12-01","count":12},{"date":"2020-12-01","count":13},{"date":"2020-12-01","count":14},{"date":"2020-12-01","count":15},{"date":"2020-12-01","count":14},{"date":"2020-12-01","count":13},{"date":"2020-12-01","count":12},{"date":"2020-12-01","count":13},{"date":"2020-12-01","count":14},{"date":"2020-12-01","count":15},{"date":"2020-12-01","count":14},{"date":"2020-12-01","count":13},{"date":"2020-12-01","count":12},{"date":"2020-12-01","count":13},{"date":"2020-12-01","count":14},{"date":"2020-12-01","count":15},{"date":"2020-12-01","count":16},{"date":"2020-12-01","count":17},{"date":"2020-12-01","count":18},{"date":"2020-12-01","count":19},{"date":"2020-12-01","count":20},{"date":"2020-12-01","count":21},{"date":"2020-12-04","count":22},{"date":"2020-12-04","count":23},{"date":"2020-12-04","count":24},{"date":"2020-12-04","count":25},{"date":"2020-12-04","count":26},{"date":"2020-12-04","count":27},{"date":"2020-12-04","count":26},{"date":"2020-12-04","count":25},{"date":"2020-12-04","count":24},{"date":"2020-12-08","count":23},{"date":"2020-12-08","count":22},{"date":"2020-12-08","count":21},{"date":"2020-12-08","count":22},{"date":"2020-12-08","count":23},{"date":"2020-12-08","count":24},{"date":"2020-12-14","count":23},{"date":"2020-12-14","count":22},{"date":"2020-12-14","count":21},{"date":"2020-12-14","count":20},{"date":"2020-12-14","count":19},{"date":"2020-12-14","count":18},{"date":"2020-12-14","count":17},{"date":"2020-12-14","count":16},{"date":"2020-12-14","count":15},{"date":"2020-12-14","count":14},{"date":"2020-12-14","count":13},{"date":"2020-12-14","count":12},{"date":"2020-12-14","count":11},{"date":"2020-12-14","count":10},{"date":"2020-12-14","count":9},{"date":"2020-12-14","count":8},{"date":"2020-12-14","count":7},{"date":"2020-12-14","count":6},{"date":"2020-12-14","count":5},{"date":"2020-12-14","count":4},{"date":"2020-12-14","count":3},{"date":"2020-12-14","count":2},{"date":"2020-12-14","count":1},{"date":"2020-12-14","count":0},{"date":"2020-12-14","count":1},{"date":"2020-12-14","count":2},{"date":"2020-12-14","count":3},{"date":"2021-01-19","count":2},{"date":"2021-01-19","count":1},{"date":"2021-01-19","count":0},{"date":"2021-01-19","count":1},{"date":"2021-01-19","count":2},{"date":"2021-01-19","count":3},{"date":"2021-02-02","count":4},{"date":"2021-02-02","count":5},{"date":"2021-02-02","count":6},{"date":"2021-02-02","count":5},{"date":"2021-02-02","count":4},{"date":"2021-02-02","count":3},{"date":"2021-02-15","count":4},{"date":"2021-02-15","count":5},{"date":"2021-02-15","count":6},{"date":"2021-02-15","count":5},{"date":"2021-02-15","count":4},{"date":"2021-02-15","count":3},{"date":"2021-03-11","count":2},{"date":"2021-03-11","count":1},{"date":"2021-03-11","count":0},{"date":"2021-03-11","count":1},{"date":"2021-03-11","count":2},{"date":"2021-03-11","count":3},{"date":"2021-03-11","count":2},{"date":"2021-03-11","count":1},{"date":"2021-03-11","count":0},{"date":"2021-03-11","count":1},{"date":"2021-03-11","count":2},{"date":"2021-03-11","count":3},{"date":"2021-03-11"

<iframe id="c73b192a-7547-4543-af40-d8c70a28c991" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2020-11-12","count":0},{"date":"2020-12-01","count":1},{"date":"2020-12-01","count":2},{"date":"2020-12-01","count":3},{"date":"2020-12-01","count":2},{"date":"2020-12-01","count":1},{"date":"2020-12-01","count":0},{"date":"2020-12-01","count":1},{"date":"2020-12-01","count":2},{"date":"2020-12-01","count":3},{"date":"2020-12-01","count":2},{"date":"2020-12-01","count":1},{"date":"2020-12-01","count":0},{"date":"2020-12-04","count":1},{"date":"2020-12-04","count":2},{"date":"2020-12-04","count":3},{"date":"2020-12-04","count":2},{"date":"2020-12-04","count":1},{"date":"2020-12-04","count":0},{"date":"2020-12-04","count":1},{"date":"2020-12-04","count":2},{"date":"2020-12-04","count":3},{"date":"2020-12-07","count":4},{"date":"2020-12-07","count":5},{"date":"2020-12-07","count":6},{"date":"2020-12-07","count":5},{"date":"2020-12-07","count":4},{"date":"2020-12-07","count":3},{"date":"2020-12-07","count":4},{"date":"2020-12-07","count":5},{"date":"2020-12-07","count":6},{"date":"2020-12-08","count":7},{"date":"2020-12-08","count":8},{"date":"2020-12-08","count":9},{"date":"2020-12-14","count":10},{"date":"2020-12-14","count":11},{"date":"2020-12-14","count":12},{"date":"2020-12-14","count":13},{"date":"2020-12-14","count":14},{"date":"2020-12-14","count":15},{"date":"2020-12-14","count":16},{"date":"2020-12-14","count":17},{"date":"2020-12-14","count":18},{"date":"2020-12-14","count":17},{"date":"2020-12-14","count":16},{"date":"2020-12-14","count":15},{"date":"2021-01-19","count":16},{"date":"2021-01-19","count":17},{"date":"2021-01-19","count":18},{"date":"2021-01-19","count":17},{"date":"2021-01-19","count":16},{"date":"2021-01-19","count":15},{"date":"2021-02-02","count":14},{"date":"2021-02-02","count":13},{"date":"2021-02-02","count":12},{"date":"2021-02-15","count":13},{"date":"2021-02-15","count":14},{"date":"2021-02-15","count":15},{"date":"2021-03-11","count":16},{"date":"2021-03-11","count":17},{"date":"2021-03-11","count":18},{"date":"2021-03-11","count":19},{"date":"2021-03-11","count":20},{"date":"2021-03-11","count":21},{"date":"2021-03-11","count":22},{"date":"2021-03-11","count":23},{"date":"2021-03-11","count":24}]},"width":800,"title":"Size of column 'Fertig' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="095c13a9-e605-45ea-9466-f35df450f461" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2020-11-30","count":0},{"date":"2020-11-30","count":1},{"date":"2020-11-30","count":2},{"date":"2020-11-30","count":3},{"date":"2020-11-30","count":4},{"date":"2020-11-30","count":5},{"date":"2020-11-30","count":6},{"date":"2020-12-04","count":7},{"date":"2020-12-04","count":8},{"date":"2020-12-04","count":9},{"date":"2020-12-04","count":10},{"date":"2020-12-04","count":11},{"date":"2020-12-04","count":12},{"date":"2021-02-02","count":11},{"date":"2021-02-02","count":10},{"date":"2021-02-02","count":9},{"date":"2021-03-12","count":10},{"date":"2021-03-12","count":11},{"date":"2021-03-12","count":12}]},"width":800,"title":"Size of column 'Resources' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="d22f4a27-c600-4029-85ba-916c491be869" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2021-02-02","count":0},{"date":"2021-02-02","count":1},{"date":"2021-02-02","count":2},{"date":"2021-02-02","count":3},{"date":"2021-02-02","count":4},{"date":"2021-02-02","count":5},{"date":"2021-02-02","count":6},{"date":"2021-02-02","count":7},{"date":"2021-02-02","count":8},{"date":"2021-02-02","count":9},{"date":"2021-02-02","count":10},{"date":"2021-02-02","count":11},{"date":"2021-02-02","count":12},{"date":"2021-02-02","count":13},{"date":"2021-02-02","count":14},{"date":"2021-02-02","count":15},{"date":"2021-02-15","count":16},{"date":"2021-02-15","count":17},{"date":"2021-02-15","count":18},{"date":"2021-02-23","count":19},{"date":"2021-02-23","count":20},{"date":"2021-02-23","count":21},{"date":"2021-03-08","count":22},{"date":"2021-03-08","count":23},{"date":"2021-03-08","count":24}]},"width":800,"title":"Size of column 'Treffen' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="b3da2d7a-4312-40cc-b1e2-9b55bd43007a" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2021-03-11","count":0},{"date":"2021-03-11","count":1},{"date":"2021-03-11","count":2},{"date":"2021-03-11","count":3}]},"width":800,"title":"Size of column 'Datensatz' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="6a0a6a25-a120-489d-b410-08aa12ec6387" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2021-03-11","count":0}]},"width":800,"title":"Size of column 'Literatur' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

board: de.tmrdlt.database.workflowlist.WorkflowList = WorkflowList(
  917L,
  "5fad0555e165558a64ad0478",
  "Masterarbeit",
  Some(""),
  None,
  0L,
  BOARD,
  Some(OPEN),
  Trello,
  None,
  2020-11-12T10:50:13,
  2021-03-12T13:44:20.759
)
data: ListBuffer[(Int, LocalDate)] = ListBuffer(
  (0, 2020-11-12),
  (1, 2020-11-12),
  (2, 2020-11-12),
  (3, 2020-11-12),
  (4, 2020-11-12),
  (5, 2020-11-12),
  (6, 2020-11-12),
  (7, 2020-11-12),
  (8, 2020-11-12),
  (9, 2020-11-12),
  (10, 2020-11-12),
  (11, 2020-11-12),
  (12, 2020-11-12),
  (13, 2020-11-30),
  (14, 2020-11-30),
  (15, 2020-11-30),
  (16, 2020-11-30),
  (17, 2020-11-30),
  (18, 2020-11-30),
  (19, 2020-11-30),
  (20, 2020-11-30),
  (21, 2020-11-30),
  (22, 2020-11-30),
  (23, 2020-11-30),
  (24, 2020-11-30),
  (25, 2020-11-30),
  (26, 2020-11-30),
  (27, 2020-11-30),
  (28, 2020-12-01),
  (29, 2020-12-01),
  (30, 2020-12-01),
  (31, 2020-12-01),
  (32, 2020-12-01),
  (33, 2020-12-01),
  (34, 2020-12-01),
  (35, 2020-12-01),